In [15]:
from datetime import datetime, timedelta
from pyspark.sql.functions import year, quarter, month, dayofmonth, hour, concat, lpad, desc
from pyspark.sql.functions import dayofweek, date_format, dayofmonth
from pyspark.sql.functions import when, floor, concat_ws
from pyspark.sql.functions import concat, col, lit, to_timestamp


StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 17, Finished, Available)

In [2]:
from pyspark.sql.functions import to_date

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 4, Finished, Available)

## **Dim Date**

In [4]:
# df = spark.sql("SELECT * FROM LTT_SilverLakehouse.orders")
# display(df.head(5))

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 6, Finished, Available)

In [3]:
# start_date, end_date = df.select(min("order_date"), max("order_date")).first()
start_date = datetime(2015, 1, 1, 0, 0)
end_date = datetime(2024, 12, 31, 23, 0)

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 5, Finished, Available)

In [5]:
# Danh sách timestamps
timestamps = []

# Tạo timestamps cho mỗi ngày và thêm vào danh sách
current_date = start_date
while current_date <= end_date:
    timestamps.append((current_date,))
    current_date += timedelta(days=1)

# Tạo DataFrame từ danh sách timestamps
timestamp_df = spark.createDataFrame(timestamps, ["date"])

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 7, Finished, Available)

In [6]:
timestamp_df.show(5)

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 8, Finished, Available)

+-------------------+
|               date|
+-------------------+
|2015-01-01 00:00:00|
|2015-01-02 00:00:00|
|2015-01-03 00:00:00|
|2015-01-04 00:00:00|
|2015-01-05 00:00:00|
+-------------------+
only showing top 5 rows



In [7]:
timestamp_df = timestamp_df.withColumn('date', to_date('date'))

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 9, Finished, Available)

In [8]:
dim_date = timestamp_df\
    .withColumn("year", year("date")) \
    .withColumn("quarter", quarter("date")) \
    .withColumn("month", month("date")) \
    .withColumn("day", dayofmonth("date"))

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 10, Finished, Available)

In [9]:
dim_date = dim_date.withColumn("date_key", concat("year", lpad("month", 2, "0"), lpad("day", 2, "0")))
dim_date = dim_date.select('date_key', 'date', 'year','quarter', 'month', 'day')

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 11, Finished, Available)

In [10]:
# Add columns with various date formats and parts
dim_date = dim_date.withColumn("full_date", date_format(dim_date["date"], "EEEE, d, MMMM yyyy"))
dim_date = dim_date.withColumn("day_of_month", dayofmonth("date")) \
                   .withColumn("day_name", date_format("date", "EEEE")) \
                   .withColumn("month_name", date_format("date", "MMMM")) \
                   .withColumn("month_abbreviation", date_format("date", "MMM"))

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 12, Finished, Available)

In [11]:
dim_date = dim_date.withColumn('date', to_date(col('date')))

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 13, Finished, Available)

In [37]:
dim_date = dim_date.withColumn('month_year', concat_ws("/", lpad(dim_date.month.cast("string"), 2, "0"), dim_date.year))


StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 39, Finished, Available)

In [41]:
display(dim_date.head(10))

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 43, Finished, Available)

SynapseWidget(Synapse.DataFrame, d2e5ae61-31eb-42e1-af84-bd5555be9471)

## **Dim Time**

In [16]:
# Tạo DataFrame chứa giờ từ 0 đến 23
hours_df = spark.range(0, 24).select(col("id").alias("hour"))

# Tạo DataFrame chứa phút từ 0 đến 59
minutes_df = spark.range(0, 60).select(col("id").alias("minute"))

# Tạo DataFrame chứa giây từ 0 đến 59
seconds_df = spark.range(0, 60).select(col("id").alias("second"))

# Tạo dim_time DataFrame bằng cách kết hợp giờ, phút và giây
dim_time = hours_df.crossJoin(minutes_df).crossJoin(seconds_df)

# Tạo cột time bằng cách kết hợp các cột hour, minute, và second
dim_time = dim_time.withColumn("time_key", 
                               concat(
                                   lpad(col("hour"), 2, "0"), 
                                   lpad(col("minute"), 2, "0"), 
                                   lpad(col("second"), 2, "0")
                               ))
# Tạo cột full time
dim_time = dim_time.withColumn("time", 
                               concat(
                                   lpad(col("hour"), 2, "0"), 
                                   lit(':'),
                                   lpad(col("minute"), 2, "0"), 
                                   lit(':'),
                                   lpad(col("second"), 2, "0")
                               ))

dim_time = dim_time.withColumn("hour_12", 
                               when(col("hour") == 12, 0)
                               .when(col("hour") < 12, col("hour"))
                               .otherwise(col("hour") - 12))

# Tạo block cho dim_time
dim_time = dim_time.withColumn("block", floor((col("hour") * 60 + col("minute").cast("int")) / 15) + 1)
# Hiển thị dim_time DataFrame
dim_time.show(5)

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 18, Finished, Available)

+----+------+------+--------+--------+-------+-----+
|hour|minute|second|time_key|    time|hour_12|block|
+----+------+------+--------+--------+-------+-----+
|   0|     0|     0|  000000|00:00:00|      0|    1|
|   0|     0|     1|  000001|00:00:01|      0|    1|
|   0|     0|     2|  000002|00:00:02|      0|    1|
|   0|     0|     3|  000003|00:00:03|      0|    1|
|   0|     0|     4|  000004|00:00:04|      0|    1|
+----+------+------+--------+--------+-------+-----+
only showing top 5 rows



In [17]:
dim_time = dim_time.select('time_key', 'time', 'hour', 'hour_12', 'minute', 'second', 'block')

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 19, Finished, Available)

In [18]:
dim_time.count()

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 20, Finished, Available)

86400

In [19]:
display(dim_time.orderBy('time'))

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1cf83e7b-2cf6-471f-9703-2e782cda1403)

In [20]:
display(dim_time)

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0ce36f36-2242-4efc-a983-d9b9199bfda8)

In [21]:
dim_time = dim_time.orderBy('hour', 'minute', 'second')

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 23, Finished, Available)

In [42]:
# dim_date.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable('LTT_GoldLakehouse.dim_date')

StatementMeta(, 073088b4-490a-4ddc-b21a-2f3191cb19b1, 44, Finished, Available)

In [23]:
dim_date.write.format("delta").mode("overwrite").saveAsTable('LTT_GoldLakehouse.dim_date')
dim_time.write.format("delta").mode("overwrite").saveAsTable('LTT_GoldLakehouse.dim_time')

StatementMeta(, 002a60f8-509b-49dc-b2b6-1df102d0f7c3, 25, Finished, Available)